In [1]:
%reload_ext autoreload
%autoreload 2



In [2]:

#クリーニングしたwebテキストをクラスタリングするモデルを作る
#このフェーズでのクラスタリングの主目的は､dedupの計算コスト(N^2)を下げること｡
# dedup用に､大きくしておくのが吉
n_clusters=10000


from sklearn.metrics import pairwise_distances
from src.cleaner.auto_cleaner import clean_text
from datasets import load_dataset
from src.load_gz import read_gzip_json_file
from sklearn.cluster import KMeans, MiniBatchKMeans
from gensim.models.fasttext import load_facebook_model
from tqdm import tqdm
from src.classify.Text2Vec import Text2Vec,texts2classes
import numpy as np
print("imported libraries")


txt="こんにちは､げんきですか?研究を "
clean_text(txt)

#gzファイルの一覧を取得
with open("temp/gz_list.txt","r") as f:
    gz_list=f.readlines()

gz_list=[i for i in gz_list if i!="\n"]
gz_list=[i.replace("\n","") for i in gz_list]
gz_list

lines=[]
path=gz_list[0]
for article in read_gzip_json_file(path):
    text = article.get('text', '')  # 'text'キーからテキストデータを取得
    lines.append(text)

# %%
lines[:3]

# %%
import random
random.shuffle(gz_list)

#クラスタリングに使うデータベースの数
n_gz=11

train_datasets=gz_list[:n_gz]

# %%
from tqdm import tqdm

#各データセットごと､N件のデータを取得
max_articles=1000
cleaned_text=[]

for path in tqdm(train_datasets):
    lines=[]
    print("loading ",path)
    for article in read_gzip_json_file(path):
        text = article.get('text', '')  # 'text'キーからテキストデータを取得
        lines.append(text)

    cnt=0
    for text in (lines):
        #text=next(dataset_iter)["text"]
        text=clean_text(text)

        if text!="":
            cleaned_text.append(text)
            cnt+=1
            if cnt>=max_articles:
                break


t2v=Text2Vec(load_facebook_model('../data/model/cc.ja.300.bin'))

# %%
title_vecs=[t2v.text2vec(i) for i in tqdm(cleaned_text)]

# %%
title_vecs=np.array(title_vecs)
# k-meansクラスタリング
print("clustering...")
kmeans = MiniBatchKMeans(n_clusters=n_clusters, random_state=1).fit(title_vecs)


# 各データポイントが割り当てられたクラスタのインデックスを取得
labels = kmeans.labels_

# 各クラスタに含まれるデータポイントの数を計算
cluster_counts = dict((i, list(labels).count(i)) for i in range(n_clusters))

print(cluster_counts)

# %%
# クラスタのセントロイドを取得
# 仮に title_vecs がデータセットの特徴ベクトル、cleaned_text がそれに対応するクリーンなテキストデータを表すとします。
# n_clusters には、生成したいクラスタの数を指定します。

centroids = kmeans.cluster_centers_

# 各クラスタに属するデータポイントのインデックスを取得
cluster_indices = {i: np.where(kmeans.labels_ == i)[0] for i in range(n_clusters)}


/home/hatakeyama/miniconda3/envs/textprocess/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


imported libraries


  0%|          | 0/11 [00:00<?, ?it/s]

loading  ../data/original_dump/oscar/oscar_ja_159.jsonl.gz


  9%|▉         | 1/11 [00:04<00:46,  4.66s/it]

...

 91%|█████████ | 10/11 [02:09<00:13, 13.23s/it]

loading  ../data/original_dump/c4/multilingual/c4-ja.tfrecord-00544-of-01024.json.gz


100%|██████████| 11000/11000 [00:27<00:00, 404.76it/s]


clustering...
{0: 1, 1: 1, 2: 0, 3: 3, 4: 1, 5: 1, 6: 3, 7: 1, 8: 2, 9: 1, 10: 2, 11: 3, 12: 4, 13: 1, 14: 1, 15: 1, 16: 1, 17: 1, 18: 2, 19: 2, 20: 2, 21: 1, 22: 1, 23: 1, 24: 1, 25: 1, 26: 2, 27: 1, 28: 1, 29: 1, 30: 1, 31: 1, 32: 2, 33: 6, 34: 1, 35: 1, 36: 1, 37: 1, 38: 1, 39: 2, 40: 1, 41: 2, 42: 1, 43: 0, 44: 1, 45: 1, 46: 1, 47: 2, 48: 1, 49: 1, 50: 1, 51: 21, 52: 2, 53: 1, 54: 1, 55: 1, 56: 1, 57: 2, 58: 1, 59: 3, 60: 2, 61: 2, 62: 1, 63: 1, 64: 1, 65: 5, 66: 1, 67: 1, 68: 1, 69: 1, 70: 3, 71: 1, 72: 1, 73: 1, 74: 5, 75: 1, 76: 1, 77: 1, 78: 1, 79: 1, 80: 0, 81: 2, 82: 1, 83: 2, 84: 1, 85: 1, 86: 1, 87: 0, 88: 3, 89: 2, 90: 2, 91: 1, 92: 1, 93: 2, 94: 2, 95: 1, 96: 2, 97: 1, 98: 1, 99: 5, 100: 2, 101: 2, 102: 1, 103: 1, 104: 2, 105: 1, 106: 1, 107: 1, 108: 1, 109: 3, 110: 1, 111: 5, 112: 3, 113: 1, 114: 1, 115: 2, 116: 1, 117: 4, 118: 2, 119: 3, 120: 1, 121: 1, 122: 1, 123: 1, 124: 1, 125: 3, 126: 5, 127: 1, 128: 0, 129: 1, 130: 7, 131: 2, 132: 1, 133: 3, 134: 1, 135: 2, 136: 1

ValueError: Found array with 0 sample(s) (shape=(0, 300)) while a minimum of 1 is required by check_pairwise_arrays.

In [4]:
"""
for cluster_id, indices in cluster_indices.items():
    # クラスタ内のデータポイントとセントロイドとの距離を計算
    distances = pairwise_distances(title_vecs[indices], [centroids[cluster_id]])
    # 距離が最も小さい上位5つのデータポイントを選択
    closest_indices = np.argsort(distances[:, 0])[:5]
    
    print(f"Cluster {cluster_id}:")
    for idx in closest_indices:
        # クラスタ内で選択されたデータポイントの実際のインデックス
        actual_idx = indices[idx]
        # テキストが長い場合は100文字に制限し、改行を削除
        short_txt=cleaned_text[actual_idx].replace('\n', '')[:100]
        print(f"{short_txt}")
    print("\n")
"""
# %%
import joblib
#save model
joblib.dump(kmeans, "../data/model/kmeans.pkl")

# %%
target_texts=cleaned_text[:10]


#print(texts2classes(target_texts,t2v,kmeans))

# %%


